In [ ]:
import pandas as pd
import os
import networkx as nx
import pickle
import sys

sys.path.append('../')
from src.utils import utils

In [ ]:
PATH = "../data/UMLS/2023AA/META/"
CHUNK_SIZE = 500000

In [ ]:

colnames = utils.extract_column_names_from_ctl_file(os.path.join(PATH, "MRCONSO.ctl"))
df_conso = utils.read_rrf_file_in_chunks(os.path.join(PATH, "MRCONSO.RRF"), CHUNK_SIZE, colnames, dtype_dict={"CUI": str})
df_conso.head()

In [ ]:
colnames = utils.extract_column_names_from_ctl_file(os.path.join(PATH, "MRHIER.ctl"))
df_hier = utils.read_rrf_file_in_chunks(os.path.join(PATH, "MRHIER.RRF"), CHUNK_SIZE, colnames, dtype_dict={"CUI": str})


In [ ]:
df_conso_sn = df_conso[df_conso['SAB'].isin(["SNOMEDCT_US", "SCTSPA"])]
scui_to_cui_dict = df_conso_sn.groupby('SCUI')['CUI'].agg(lambda x: list(set(x))).to_dict()
aui_to_cui_dict = df_conso_sn.set_index('AUI')['CUI'].to_dict()
df_hier['CUI1'] = df_hier['PAUI'].map(aui_to_cui_dict).dropna()
df_hier.rename(columns={'CUI': 'CUI2'}, inplace=True)
df_parent_child = df_hier[['CUI1', 'CUI2']].dropna().drop_duplicates()
df_parent_child.head()

In [ ]:
df_conso_sn_esp = df_conso_sn[df_conso_sn.LAT == "SPA"]

In [ ]:
grouped = df_conso_sn_esp.groupby('CUI')['STR'].agg(lambda x: list(set([str for str in x if str.strip() != '']))).reset_index()

G = nx.DiGraph()

for index, row in grouped.iterrows():
    G.add_node(row['CUI'], name=row['STR'])

for index, row in df_parent_child.iterrows():
    G.add_edge(row['CUI1'], row['CUI2'])


In [ ]:
with open('scui_to_cui_dict.pkl', 'wb') as f:
    pickle.dump(scui_to_cui_dict, f)

with open('graph_G.pkl', 'wb') as f:
    pickle.dump(G, f)